In [29]:
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from tqdm import tqdm

In [2]:
# Device configuration (use GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the number of output classsed (e.g., 10 for CIFAR-10)
num_classes = 10

# Load the pre-trained ResNet18 model
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Freeze all layers in the feature extractor
for param in model.parameters():
    param.requires_grad = False

In [3]:
# Replace the final fully connected layer for fine-tuning
# ResNet18 has an fc layer with 512 input features, we replace it with a new one
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move the model to the configured device (CPU or GPU)
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

In [9]:
# Load your custom dataset
# For example, using CIFAR-10 dataset with augmentations
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=True)

100%|██████████| 170498071/170498071 [00:22<00:00, 7490033.06it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [47]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = outputs.max(dim=1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
    # Print statistics for each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss : {running_loss/total:.4f}, Accuracy: {100* correct/total:.2f}%')


Epoch [1/10], Loss : 0.0407, Accuracy: 54.59%
Epoch [2/10], Loss : 0.0376, Accuracy: 57.89%
Epoch [3/10], Loss : 0.0372, Accuracy: 58.60%
Epoch [4/10], Loss : 0.0367, Accuracy: 58.98%
Epoch [5/10], Loss : 0.0368, Accuracy: 59.00%
Epoch [6/10], Loss : 0.0365, Accuracy: 59.27%
Epoch [7/10], Loss : 0.0365, Accuracy: 59.11%
Epoch [8/10], Loss : 0.0366, Accuracy: 59.16%
Epoch [9/10], Loss : 0.0363, Accuracy: 59.53%
Epoch [10/10], Loss : 0.0359, Accuracy: 59.78%


In [48]:
# Testing the model (evaluation)
model.eval() 
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print(f'Test Accuracy: {100 * correct/total:.2f}%')

Test Accuracy: 75.29%


In [49]:
torch.save(model.state_dict(), 'fine_tuned_resnet.pth')